In [ ]:
import pydicom as dm
import pylab
#import cv2
import pylab
import numpy as np
import pylab as pl
import scipy.misc 
import numpy as np
import matplotlib.pyplot as plt
import glob
from skimage import data, color
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter
from skimage.util import img_as_ubyte
from skimage.util import crop, pad     


In [ ]:
hough_radii = np.arange(20, 35, 2)


In [ ]:
hough_radii

# Patient id 

In [ ]:
n=429

In [ ]:
def Patient_slice_get_sax_slice(path,patient):
    fullpath=str(path)+str(patient)+'/study/sax_*/'
    L=glob.glob(fullpath)
    F=[]
    for i in L:
        F.append(i.split('_')[1].split('/')[0])
    return F

In [ ]:
def Get_images_path_patient_sax_slice(path,patient,saxslice):
        fullpath=str(path)+str(patient)+'/study/sax_'+saxslice+'/*'
        L=glob.glob(fullpath)
        return L

In [ ]:
def fourier_transform(L):
    ds = dm.read_file(L[0])
    finalArray = np.zeros((ds.pixel_array.shape[0], ds.pixel_array.shape[1], len(L)))
    for index, image in enumerate(L):
        ds = dm.read_file(L[index])
        toFill = ds.pixel_array
        finalArray[:, :, index] = toFill
    ff=np.fft.fft(finalArray)
    return ff

In [ ]:
def harmonic_number(L,n):
    B=fourier_transform(L)
    harmonic_n = np.abs(B[:,:,n])
    pylab.imshow(harmonic_n)
    pylab.show() 
    return  harmonic_n  
    

In [ ]:
def crop_image(  image,  crop_mask  ):
    
    ''' Crop the non_zeros pixels of an image  to a new image 
         
         
    '''
    pxlst = np.where(crop_mask.ravel())[0]
    dims = crop_mask.shape
    imgwidthy = dims[1]   #dimension in y, but in plot being x
    imgwidthx = dims[0]   #dimension in x, but in plot being y
    #x and y are flipped???
    #matrix notation!!!
    pixely = pxlst%imgwidthy
    pixelx = pxlst//imgwidthy

    minpixelx = np.min(pixelx)
    minpixely = np.min(pixely)
    maxpixelx = np.max(pixelx)
    maxpixely = np.max(pixely) 
    crops = crop_mask*image
    img_crop = crop(  crops, ((minpixelx, imgwidthx -  maxpixelx -1 ),
                                (minpixely, imgwidthy -  maxpixely -1 )) )
    return img_crop 

In [ ]:
def high_variance(L):
    x=0
    y=0
    for i in L:
        ds = dm.read_file(i)
        x=ds.pixel_array+x
    x=x/(len(L)) 
    for j in L:
        ds = dm.read_file(j)
        y=(ds.pixel_array-x)**2+y
        y=y/(len(L))
    return y   

In [ ]:
G=Patient_slice_get_sax_slice('/Users/egouda/Downloads/train/',n)

In [ ]:
 D=[]
E=[]
for i in G:
    print (i)
    LL=Get_images_path_patient_sax_slice('/Users/egouda/Downloads/train/',n,i)
    A=harmonic_number(LL,1)
    B=high_variance(LL)
    D.append(A)
    E.append(B)
    
    

In [ ]:
F=[]

In [ ]:
for i in range(len(D)):
    x=D[i]+E[i]
    F.append(x)          
               

In [ ]:
for t in F:
    pylab.imshow(t)
    pylab.show()
    

In [ ]:
Total_final_array=np.zeros((F[1].shape[0],F[1].shape[1]))
Total_final_array_Fourier = np.zeros((D[1].shape[0],D[1].shape[1]))
Total_final_array_Varience=np.zeros((E[1].shape[0],E[1].shape[1]))

In [ ]:
print (Total_final_array_Fourier.shape)
print (Total_final_array_Varience.shape)
print (Total_final_array.shape)

In [ ]:
# kernal=np.ones((5,5),'uint8')
# for i in F:
#     result = cv2.GaussianBlur(i, (5, 5), 0)
#     erode=cv2.erode(result,kernal,iterations=5)
#     dilate=cv2.dilate(erode,kernal,iterations=5)
#     Total_final_array=Total_final_array+dilate
    

In [ ]:
Total_final_array_Varience=np.zeros((E[1].shape[0],E[1].shape[1]))
for i in E:
    #result = cv2.GaussianBlur(i, (5, 5), 0)
    #erode=cv2.erode(result,kernal,iterations=5)
    #dilate=cv2.dilate(erode,kernal,iterations=5)
    Total_final_array_Varience=Total_final_array_Varience+i

In [ ]:
Total_final_array_Fourier=np.zeros((E[1].shape[0],E[1].shape[1]))
for i in D:
    #result = cv2.GaussianBlur(i, (5, 5), 0)
    #erode=cv2.erode(result,kernal,iterations=5)
    #dilate=cv2.dilate(erode,kernal,iterations=5)
    Total_final_array_Fourier=Total_final_array_Fourier+i

In [ ]:
pylab.imshow(Total_final_array_Fourier)
pylab.show() 

In [ ]:
pylab.imshow(Total_final_array_Varience)
pylab.show() 

In [ ]:
Total_final_array=Total_final_array_Varience+Total_final_array_Fourier

In [ ]:
pylab.imshow(Total_final_array)
pylab.show()

In [ ]:
for j in E:
    pylab.imshow(j)
    pylab.show() 
    

In [ ]:
# Load picture and detect edges
V=[]
for i in F:   
    image = i
    edges = canny(image, sigma=3, low_threshold=10, high_threshold=100)


    # Detect two radii
    hough_radii = np.arange(20, 35, 2)
    hough_res = hough_circle(edges, hough_radii)

    # Select the most prominent 5 circles
    accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii,
                                               total_num_peaks=5)

    # Draw them
    #fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4))
    image = color.gray2rgb(image)
    for center_y, center_x, radius in zip(cy, cx, radii):
        circy, circx = circle_perimeter(center_y, center_x, radius)
        image[circy, circx] = (220, 20, 20)
    #ax.imshow(image)
    #plt.show()
    imagex=image[:,:,1]-image[:,:,0]
    V.append(imagex)

In [ ]:
Total_final_array_H=np.zeros((F[1].shape[0],F[1].shape[1]))
for i in V:
    Total_final_array_H=Total_final_array_H+i
    pylab.imshow(i)
    pylab.show()


In [ ]:
Total_final_array_H

In [ ]:
edges = canny( Total_final_array_H, sigma=3, low_threshold=10, high_threshold=20)
height=Total_final_array_H.shape[0]
width=Total_final_array_H.shape[1]
binary = np.zeros([height,width],'uint8')
for row in range(0,height):
    for col in range(0, width):
        if edges.sum(axis=0)[col]*edges.sum(axis=1)[row]>0 :
            binary[row][col]=1

In [ ]:
# Bx=[]
# for i in V:
#     edges = canny(i, sigma=3, low_threshold=10, high_threshold=20)
#     Bx.append(edges)
#     pylab.imshow(edges)
#     pylab.show()

In [ ]:
# for i in Bx:
#     pylab.imshow(i)
#     pylab.show()
    

In [ ]:
# binary = np.zeros([height,width],'uint8')
# for i in Bx:
#     binary=binary+i
#     pylab.imshow(binary)
#     pylab.show()
    

In [ ]:
# edges = canny(binary, sigma=3, low_threshold=10, high_threshold=10)

In [ ]:
# pylab.imshow(binary)
# pylab.show()

In [ ]:
pylab.imshow(binary)
pylab.show()

In [ ]:
from skimage import measure
contours = measure.find_contours(binary,0.8)

In [ ]:
contours[1].shape

In [ ]:
fig, ax = plt.subplots()
ax.imshow(binary, interpolation='nearest', cmap=plt.cm.gray)

for n, contour in enumerate(contours):
    ax.plot(contour[:, 1], contour[:, 0], linewidth=2)

ax.axis('image')
ax.set_xticks([])
ax.set_yticks([])
plt.show()


In [ ]:
X=[]
Y=[]
for i in G:
    LL=Get_images_path_patient_sax_slice('/Users/egouda/Downloads/train/',n,i)
    for j in LL:
        ds = dm.read_file(j)
        t=ds.pixel_array
        c=crop_image(t,binary)
        d=t*binary
        X.append(c)
        Y.append(d)

In [ ]:
for i in Y:
    pylab.imshow(i,cmap=pylab.cm.bone)
    pylab.show()

In [ ]:
for i in X:
    pylab.imshow(i,cmap=pylab.cm.bone)
    pylab.show()

In [ ]:
# Load picture and detect edges
V=[]
for i in X:   
    image = i
    edges = canny(image, sigma=3, low_threshold=10, high_threshold=100)


    # Detect two radii
    hough_radii = np.arange(20, 35, 2)
    hough_res = hough_circle(edges, hough_radii)

    # Select the most prominent 5 circles
    accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii,
                                               total_num_peaks=5)

    # Draw them
    #fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4))
    image = color.gray2rgb(image)
    for center_y, center_x, radius in zip(cy, cx, radii):
        circy, circx = circle_perimeter(center_y, center_x, radius)
        #image[circy, circx] = (220, 20, 20)
    #ax.imshow(image)
    #plt.show()
    imagex=image[:,:,1]-image[:,:,0]
    V.append(imagex)

In [ ]:
len(V)

In [ ]:
def dislplay_list(L):
    for i in L:
        pylab.imshow(i)
        pylab.show()
        

In [ ]:
dislplay_list(V)

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
X =np.zeros((ds.pixel_array.shape[0], ds.pixel_array.shape[1], 30))
pca = PCA(n_components=1)
pca.componrnt = True
newX = pca.fit_transform(X)
print (newX)

In [ ]:
len(D)

In [ ]:
Bx=[]
for i in D:
    x=i.flatten()
    Bx.append(x)

In [ ]:
i.shape

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(Bx)


In [ ]:
df.transpose().values

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
X =df.transpose().values
pca = PCA(n_components=1)
pca.componrnt = True
newX = pca.fit_transform(X)
print (newX)

In [ ]:
newX

In [ ]:
img_PCA=newX.reshape((180, 256))*newX.reshape((180, 256))

In [ ]:
np.log(img_PCA)

In [ ]:
pylab.imshow(np.log(img_PCA)*np.log(img_PCA)*np.log(img_PCA))
pylab.show()

In [ ]:
PCA_image=np.log(img_PCA)*np.log(img_PCA)*np.log(img_PCA)


In [ ]:
# Load picture and detect edges
image = PCA_image
edges = canny(image, sigma=3, low_threshold=10, high_threshold=100)


# Detect two radii
hough_radii = np.arange(20, 35, 2)
hough_res = hough_circle(edges, hough_radii)

# Select the most prominent 5 circles
accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii,
                                           total_num_peaks=5)

# Draw them
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4))
image = color.gray2rgb(image)
for center_y, center_x, radius in zip(cy, cx, radii):
    circy, circx = circle_perimeter(center_y, center_x, radius)
    image[circy, circx] = (220, 20, 20)
#ax.imshow(image)
#plt.show()

In [ ]:
imagex=image[:,:,1]-image[:,:,0]

In [ ]:
pylab.imshow(imagex)
pylab.show()

In [ ]:
edges = canny(imagex, sigma=3, low_threshold=10, high_threshold=50)

In [ ]:
height=imagex.shape[0]
width=imagex.shape[1]
binary = np.zeros([height,width],'uint8')


for row in range(0,height):
    for col in range(0, width):
        if Total_final_array[row][col]*edges.sum(axis=0)[col]*edges.sum(axis=1)[row]>0 :
            binary[row][col]=1

In [ ]:
pylab.imshow(binary)
pylab.show()

In [ ]:

import matplotlib.pyplot as plt

from skimage import measure




# Find contours at a constant value of 0.8
contours = measure.find_contours(imagex, 0.8)

# Display the image and plot all contours found
fig, ax = plt.subplots()
ax.imshow(imagex, interpolation='nearest', cmap=plt.cm.gray)

for n, contour in enumerate(contours):
    ax.plot(contour[:, 1], contour[:, 0], linewidth=2)

ax.axis('image')
ax.set_xticks([])
ax.set_yticks([])
plt.show()

In [ ]:
X=[]
for i in G:
    LL=Get_images_path_patient_sax_slice('/Users/egouda/Downloads/train/',n,i)
    for j in LL:
        ds = dm.read_file(j)
        t=ds.pixel_array
        c=crop_image(t,binary)
        X.append(c)

In [ ]:
for i in X:
    pylab.imshow(i,cmap=pylab.cm.bone)
    pylab.show()

In [ ]:
P=[]
for i in G:
    LL=Get_images_path_patient_sax_slice('/Users/egouda/Downloads/train/',n,i)
    for j in LL:
            ds = dm.read_file(j)
            x=ds.pixel_array.flatten()
            P.append(x)

In [ ]:
n

In [ ]:
ds.pixel_array.shape

In [ ]:
len(P)

In [ ]:
df = pd.DataFrame(P)

In [ ]:
df

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
X =df.transpose().values
pca = PCA(n_components=1)
pca.componrnt = True
newX = pca.fit_transform(X)
print (newX)

In [ ]:
newX.shape

In [ ]:
img_PCA=newX.reshape((256, 230))

In [ ]:
(img_PCA)**2

In [ ]:
TT=np.log(img_PCA)


In [ ]:
pylab.imshow(TT)
pylab.show()

In [ ]:
D=[]
E=[]
for i in G:
    print i
    LL=Get_images_path_patient_sax_slice('/Users/egouda/Downloads/train/',n,i)
    A=harmonic_number(LL,1)
    B=high_variance(LL)
    D.append(A)
    E.append(B)

In [ ]:
Total_final_array_PCA=np.zeros((D[1].shape[0],D[1].shape[1]))
for i in D:
    #result = cv2.GaussianBlur(i, (5, 5), 0)
    #erode=cv2.erode(result,kernal,iterations=5)
    #dilate=cv2.dilate(erode,kernal,iterations=5)
    Total_final_array_PCA=Total_final_array_PCA+i

In [ ]:
pylab.imshow(TT)
pylab.show()

In [ ]:
pylab.imshow(Total_final_array_PCA-TT)
pylab.show()